In [106]:
# import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm


In [107]:
# import CADS dataset
filepath = r"C:\Users\marti\Documents\GitHub\481MidtermPaper\CADS.csv"
df = pd.read_csv(filepath)
df.head()

,PUMFID,REFYEAR,REFMONTH,PROV,DVURBAN,MHHTYPEB,SEX,DV_AGEP,HHI_01,HDSIZE,...,ANY15DLF,IL6DL,IL5DL,DHAR12,ASSISTOD,DVSST1,DVSS1,CIG30,WEIGHTP,VERDATE
0,1000,2019,12,46,2,3,1,8,1,2,...,2,2,2,2,6,2,2,2,1256.4374,21DEC2020
1,1001,2019,9,46,2,6,1,5,1,4,...,2,2,2,2,6,3,3,2,1576.0206,21DEC2020
2,1002,2019,6,35,1,3,2,8,1,2,...,2,2,2,2,6,3,3,2,3880.9735,21DEC2020
3,1003,2019,8,11,1,3,1,8,1,3,...,1,1,2,2,6,1,1,1,212.8512,21DEC2020
4,1004,2019,6,12,1,3,1,9,1,3,...,1,1,1,6,6,2,2,2,1078.1501,21DEC2020


In [108]:
df['DV_AGEP'].unique()

array([8, 5, 9, 6, 7, 1, 3, 2, 4], dtype=int64)

In [109]:
# reduce df to columns of interest
sub_cols = ['SEX','PROV','DV_AGEP','DEM_05P','DEM_20','DEM_30','AB_15','INC_50','DVURBAN','HWB_10','HWB_05','ALC_20P','ALC_55','ALC_120A','AL_120BP',
            'ALC_90C','ALC_90D','ALC_90B','CAN_20A','CAN_45A','CAN_60','CAN_145','CAN_125A','C_125BP','CAN_80','CAN_15P','CAN_112C','CANALCEV',
            'ANY14DYR','TT_20','TT_25A','SS_05','SS_10','SS_25']

df = df[sub_cols]

# set up provincial controls
df['prov_BC'] = (df.loc[:,'PROV'] == 59).astype(int)
df['prov_AB'] = (df.loc[:,'PROV'] == 48).astype(int)
df['prov_SK'] = (df.loc[:,'PROV'] == 47).astype(int)
df['prov_MB'] = (df.loc[:,'PROV'] == 46).astype(int)
df['prov_ON'] = (df.loc[:,'PROV'] == 35).astype(int)
df['prov_QC'] = (df.loc[:,'PROV'] == 24).astype(int)
df['prov_NB'] = (df.loc[:,'PROV'] == 13).astype(int)
df['prov_NS'] = (df.loc[:,'PROV'] == 12).astype(int)
df['prov_PE'] = (df.loc[:,'PROV'] == 11).astype(int)
df['prov_NL'] = (df.loc[:,'PROV'] == 10).astype(int)

# sex controls
df['male'] = (df.loc[:,'SEX'] == 1).astype(int)

# age controls
df['age15_24'] = (df.loc[:,'DV_AGEP'] == any([1,2,3,4])).astype(int)
df['age25_34'] = (df.loc[:,'DV_AGEP'] == 5).astype(int)
df['age35_44'] = (df.loc[:,'DV_AGEP'] == 6).astype(int)
df['age45_54'] = (df.loc[:,'DV_AGEP'] == 7).astype(int)
df['age55_64'] = (df.loc[:,'DV_AGEP'] == 8).astype(int)
df['age65_plus'] = (df.loc[:,'DV_AGEP'] == 9).astype(int)

# rural/urban divide
df = df.loc[df['DVURBAN'] != 9] # drop observations where urban/rural is unstated (code 9)
df['urban'] = (df.loc[:,'DVURBAN'] == 1).astype(int)

# Indigenous identity
df['indigenous'] = (df.loc[:,'AB_15'] == 1).astype(int)

# Highest education
df['no_HS'] = (df.loc[:,'DEM_20'] == 1).astype(int)
df['HS'] = (df.loc[:,'DEM_20'] == 2).astype(int)
df['bel_BA'] = (df.loc[:,'DEM_20'] == any([3,4,5])).astype(int) # Post-secondary completed incl. trades, below Bachelor's
df['BA'] = (df.loc[:,'DEM_20'] == 6).astype(int) # Bachelor's degree
df['abv_BA'] = (df.loc[:,'DEM_20'] == 7).astype(int)
df = df.loc[df['DEM_20'] != 99]

# worked last week at job or business
df['worked'] = (df.loc[:,'DEM_30'] == 1).astype(int)

# Household income
df['inc_bel_20'] = (df.loc[:,'INC_50'] == 1).astype(int)
df['inc20_40'] = (df.loc[:,'INC_50'] == 2).astype(int)
df['inc40_60'] = (df.loc[:,'INC_50'] == 3).astype(int)
df['inc60_80'] = (df.loc[:,'INC_50'] == 4).astype(int)
df['inc80_100'] = (df.loc[:,'INC_50'] == 5).astype(int)
df['inc100_150'] = (df.loc[:,'INC_50'] == 6).astype(int)
df['inc_abv_150'] = (df.loc[:,'INC_50'] == 7).astype(int)

# lists of controls
c_income = ['inc_bel_20','inc20_40','inc40_60','inc60_80','inc80_100','inc100_150','inc_abv_150']
c_edu = ['no_HS','HS','bel_BA','BA','abv_BA']
c_age = ['age25_34','age35_44','age45_54','age55_64','age65_plus']
c_prov = ['prov_BC','prov_AB','prov_SK','prov_MB','prov_ON','prov_QC','prov_NB','prov_NS','prov_PE','prov_NL']

# add constant
df = sm.add_constant(df)

In [110]:
df['ALC_120A'].unique()

array([2, 6, 1, 9], dtype=int64)

In [123]:
# 1. How do drinking/driving and weed/driving rates differ by urban/rural?

cols = []
for column in [c_income, c_age, c_edu, c_prov]:
    for cat in column:
        cols.append(cat)

cols.append('const')
cols.append('worked')
cols.append('indigenous')
cols.append('urban')
cols.append('ALC_120A')
cols.append('CAN_125A')

df2 = df[cols]
df2 = df2.rename(columns={'ALC_120A': 'drinkDriving', 'CAN_125A': 'weedDriving'})


df2_drink = df2.loc[(df2['drinkDriving'] == 1) | (df2['drinkDriving'] == 2)]
df2_cann = df2.loc[(df2['weedDriving'] == 1) | (df2['weedDriving'] == 2)]

df2_drink.loc[:,'drinkDriving'] = (df2.loc[:,'drinkDriving'] == 1).astype(int)


# logit regression: drinkDriving ~ urban
logit = sm.Logit(df2_drink['drinkDriving'], df2_drink['urban']).fit()
print(logit.summary())

# 2nd logit regression: drinkDriving ~ urban + education controls
exog = ['const','urban'] # set up regressors
for i in c_edu[1:]:
    exog.append(i)
      
logit2 = sm.Logit(df2_drink['drinkDriving'], df2_drink[exog]).fit()
print(logit2.summary())

# 3rd logit regression: drinkDriving ~ urban + education controls
exog = ['const','urban'] # set up regressors
for i in c_income[1:]:
    exog.append(i)
      
logit3 = sm.Logit(df2_drink['drinkDriving'], df2_drink[exog]).fit()
print(logit3.summary())

# 3rd logit regression: drinkDriving ~ urban + provincial FE's
exog = ['const','urban'] # set up regressors
for i in c_prov[1:]:
    exog.append(i)
      
logit4 = sm.Logit(df2_drink['drinkDriving'], df2_drink[exog]).fit()
print(logit4.summary())

Optimization terminated successfully.
         Current function value: 0.353197
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:           drinkDriving   No. Observations:                 6660
Model:                          Logit   Df Residuals:                     6659
Method:                           MLE   Df Model:                            0
Date:                Sun, 15 Oct 2023   Pseudo R-squ.:                 -0.6020
Time:                        14:27:59   Log-Likelihood:                -2352.3
converged:                       True   LL-Null:                       -1468.4
Covariance Type:            nonrobust   LLR p-value:                       nan
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
urban         -2.7209      0.059    -45.878      0.000      -2.837      -2.605
Optimization terminated succe

C:\Users\marti\AppData\Local\Temp\ipykernel_22208\1357133101.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_drink.loc[:,'drinkDriving'] = (df2.loc[:,'drinkDriving'] == 1).astype(int)
C:\Users\marti\AppData\Local\Temp\ipykernel_22208\1357133101.py:22: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df2_drink.loc[:,'drinkDriving'] = (df2.loc[:,'drinkDriving'] == 1).astype(int)


In [118]:
for col in exog:
    print(col + ':' + str(sum(df2_drink[col])))

const:6660.0
urban:4907
no_HS:392
HS:1366
bel_BA:392
BA:1470


In [113]:
# logit regression: weedDriving ~ urban

# 2nd logit regression: weedDriving ~ urban + controls

In [120]:
df2_drink['drinkDriving'].unique()

array([0, 1])